# Función para generar GeoTiff


In [2]:
import xarray as xr
import numpy as np
import pangaea as pa
import xarray as xr

In [3]:
def corregir_wrfout(ruta_wrfout):
    xds =  xr.open_dataset(ruta_wrfout)
    variables = ['XLAT', 'XLONG', 'XLAT_U', 'XLONG_U', 'XLAT_V', 'XLONG_V']
    for var in variables:
        if len(xds.coords['XLAT'].shape) > 2:
            xds.coords[var] = xds.coords[var].mean(axis=0)
    xds.to_netcdf(ruta_wrfout + '.nc')
    xds.close()

    
def abrir_plsm(ruta_wrfout):
    plsm = pa.open_mfdataset(ruta_wrfout,
                       lat_var='XLAT',
                       lon_var='XLONG',
                       time_var='XTIME',
                       lat_dim='south_north',
                       lon_dim='west_east',
                       time_dim='Time')
    return plsm


def to_projection(plsm, variable, projection=None):
    """Convert Grid to New Projection.
        Parameters
        ----------
        variable: :obj:`str`
            Name of variable in dataset.
        projection: :func:`osr.SpatialReference`
            Projection to convert data to.
        Returns
        -------
        :func:`xarray.Dataset`
    """
    if projection == None:
        from osgeo import osr, gdalconst
        projection = osr.SpatialReference()
        projection.ImportFromProj4("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    new_data = []
    from gazar.grid import ArrayGrid
    for band in range(plsm._obj.dims[plsm.time_dim]):
        arr_grid = ArrayGrid(in_array=plsm._obj[variable][band].values[::-1,:],
                             wkt_projection=plsm.projection.ExportToWkt(),
                             geotransform=plsm.geotransform)
        ggrid = arr_grid.to_projection(projection, gdalconst.GRA_Average)
        new_data.append(ggrid.np_array())

    plsm.to_datetime()
    return plsm._export_dataset(variable, np.array(new_data),
ggrid)


def guardar_tif(vari, arr, out_path):
    import rasterio
    new_dataset = rasterio.open(out_path, 'w', driver='GTiff',
                                height = arr.shape[0], width = arr.shape[1],
                                count=1, dtype=str(arr.dtype),
                                crs=vari.lsm.projection.ExportToWkt(),
                                transform=vari.lsm.affine)
    new_dataset.write(arr, 1)
    new_dataset.close()

    
def convertir_variable(plsm, variable):
    vari = to_projection(plsm.lsm, variable)
    vari['lat'] = vari['lat'].sel(x=1)
    vari['lon'] = vari['lon'].sel(y=1)
    vari = vari.rename({'lat': 'y', 'lon': 'x'})
    return vari
    

def genear_tif(ruta_wrfout, variable, time_idx, out_path):
    plsm = abrir_plsm(ruta_wrfout)
    vari = convertir_variable(plsm, variable)
    arr = vari[variable][time_idx].values[:,:]
    arr[arr == 0] = np.nan 
    guardar_tif(vari, arr, out_path)
    plsm.close()


def genear_tif_prec(ruta_wrfout, out_path=None):
    plsm = abrir_plsm(ruta_wrfout)
    if out_path == None:
        out_path = 'ppn_' + plsm.START_DATE[:-6] + '.tif'
    plsm.variables['RAINNC'].values = plsm.variables['RAINNC'].values + 1000
    plsm.variables['RAINC'].values = plsm.variables['RAINC'].values + 1000
    rainnc = convertir_variable(plsm, 'RAINNC')
    rainc = convertir_variable(plsm, 'RAINC')
    arr_33 = rainnc.RAINNC[33].values[:,:] + rainc.RAINC[33].values[:,:]
    arr_9 = rainnc.RAINNC[9].values[:,:] + rainc.RAINC[9].values[:,:]
    arr = arr_33 - arr_9
    arr[arr_9 == 0] = np.nan 
    guardar_tif(rainnc, arr, out_path)
    plsm.close()

In [4]:
#corregir_wrfout('/home/andres/wrfout_d01_2018-10-18_06:00:00')

In [85]:
genear_tif('wrfout.nc', 'T2', 15, 't2_15-3.tif')

/home/andres/anaconda3/envs/gpd/lib/python3.6/site-packages/xarray/backends/api.py:611: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in xarray v0.12. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  datasets = [open_(p, **open_kwargs) for p in paths]
/home/andres/anaconda3/envs/gpd/lib/python3.6/site-packages/pangaea/read.py:140: FutureWarning: The inplace argument has been deprecated and will be removed in xarray 0.12.0.
  inplace=True


In [5]:
genear_tif_prec('wrfout.nc')

/home/andres/anaconda3/envs/gpd/lib/python3.6/site-packages/xarray/backends/api.py:611: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in xarray v0.12. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  datasets = [open_(p, **open_kwargs) for p in paths]
/home/andres/anaconda3/envs/gpd/lib/python3.6/site-packages/pangaea/read.py:140: FutureWarning: The inplace argument has been deprecated and will be removed in xarray 0.12.0.
  inplace=True
